In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Raising an AnalysisError

This example demonstrates the effect of raising an `AnalysisError` in your Component's `compute` function.  The result depends on which driver and optimizer is used.  The `SNOPT` and `IPOPT` optimizers, used in conjunction with [pyOptSparseDriver](../../features/building_blocks/drivers/pyoptsparse_driver.ipynb), are good options if your model has invalid regions.


## Model


For this somewhat contrived case, we will assume some range of input values to our Component is invalid and raise an `AnalysisError` if those inputs are encountered.  We will use the [Paraboloid](../../basic_user_guide/single_disciplinary_optimization/first_analysis) as the basis for our example, modifying it so that it will raise an AnalysisError if the x or y inputs are within a specified range.

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_paraboloid_invalid_region", get_code("openmdao.test_suite.components.paraboloid_invalid_region.Paraboloid"), display=False)

:::{Admonition} `Paraboloid` class definition 
:class: dropdown

{glue:}`code_paraboloid_invalid_region`
:::

First, we will define a function to create a Problem instance while allowing us to specify the optimizer and the invalid region:

In [ ]:
import openmdao.api as om

from openmdao.test_suite.components.paraboloid_invalid_region import Paraboloid


def setup_problem(optimizer, invalid_x=None, invalid_y=None):
    # Paraboloid model with optional AnalysisErrors
    model = om.Group()

    model.add_subsystem('p1', om.IndepVarComp('x', 50.0), promotes=['*'])
    model.add_subsystem('p2', om.IndepVarComp('y', 50.0), promotes=['*'])

    comp = model.add_subsystem('comp',
                               Paraboloid(invalid_x, invalid_y),
                               promotes=['*'])

    model.add_subsystem('con', om.ExecComp('c = - x + y'), promotes=['*'])

    model.add_design_var('x', lower=-50.0, upper=50.0)
    model.add_design_var('y', lower=-50.0, upper=50.0)

    model.add_objective('f_xy')
    model.add_constraint('c', upper=-15.)

    # pyOptSparseDriver with selected optimizer
    driver = om.pyOptSparseDriver(optimizer=optimizer)
    if optimizer == 'IPOPT':
        driver.opt_settings['file_print_level'] = 5
    driver.options['print_results'] = False
    driver.options['output_dir'] = None  # will put the optimizer output file in the current directory

    # setup problem & initialize values
    prob = om.Problem(model, driver)
    prob.setup()

    prob.set_val('x', 50)
    prob.set_val('y', 50)

    return prob, comp

## Example using IPOPT

First we will run the Paraboloid optimization as normal, without raising any errors. In doing this, we can see the nominal path that the optimizer follows throught solution space to arrive at the optimum.  For this initial case, we will use the `IPOPT` optimizer:

In [ ]:
prob, comp = setup_problem('IPOPT')
prob.run_driver()

for (x, y, f_xy) in comp.eval_history:
    print(f"x: {x:9.5f}  y: {y:9.5f}  f_xy: {f_xy:10.5f}")

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal
assert_near_equal(prob['x'], 7.166667, 1e-6)
assert_near_equal(prob['y'], -7.833334, 1e-6)

Now, we will define our invalid region as `x` between 7.2 and 10.2 and `y` between -50 and -10.  This region was chosen as it is crossed in the course of the nominal optimization from our chosen starting point at `x=50`, `y=50`.

In [ ]:
invalid_x = (7.2, 10.2)
invalid_y = (-50., -40.)

We will recreate the problem using this invalid region and see that the optimizer's path to the optimum now must reroute around the invalid values. It will take many more iterations to get to the solution, but IPOPT still gets there in the end:

In [ ]:
prob, comp = setup_problem('IPOPT', invalid_x, invalid_y)
prob.run_driver()

for i, (x, y, f_xy) in enumerate(comp.eval_history):
    print(f"{i:2d}  x: {x:9.5f}  y: {y:9.5f}  f_xy: {f_xy:10.5f}")

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal
assert_near_equal(prob['x'], 7.166667, 1e-6)
assert_near_equal(prob['y'], -7.833334, 1e-6)

We can see how many times our Component raised an AnalysisError and at which iteration they occurred:

In [ ]:
print(f"Number of errors: {len(comp.raised_eval_errors)}")
print(f"Iterations:{comp.raised_eval_errors}")


Looking at the IPOPT output file (`IPOPT.out`) will reveal what happened when the optimizer encountered these bad points. Here we just show a relevant subsection of the file:

In [ ]:
with open("IPOPT.out", encoding="utf-8") as f:
    IPOPT_history = f.read()
beg = IPOPT_history.find("iter    objective")
end = IPOPT_history.find("(scaled)", beg)
print(IPOPT_history[beg:end])

Specifically, we can see the following message when IPOPT changes its search in response to the bad point:

    Warning: Cutting back alpha due to evaluation error

In [ ]:
count = 0

for line in IPOPT_history.split('\n'):
    if 'Cutting back alpha' in line:
        print(line)
        count = count + 1

print("\nNumber of times IPOPT encountered an evaluation error:", count)

## Example using SNOPT

We can exercise the same model using `SNOPT` as our optimizer, with similar results. First we will run the nominal case, and then again with the invalid region:

In [ ]:
prob, comp = setup_problem('SNOPT')
prob.run_driver()

for i, (x, y, f_xy) in enumerate(comp.eval_history):
    print(f"{i:2d}  x: {x:9.5f}  y: {y:9.5f}  f_xy: {f_xy:10.5f}")

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal
assert_near_equal(prob['x'], 7.166667, 1e-6)
assert_near_equal(prob['y'], -7.833334, 1e-6)

In [ ]:
prob, comp = setup_problem('SNOPT', invalid_x, invalid_y)
prob.run_driver()

for i, (x, y, f_xy) in enumerate(comp.eval_history):
    print(f"{i:2d}  x: {x:9.5f}  y: {y:9.5f}  f_xy: {f_xy:10.5f}")

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal
assert_near_equal(prob['x'], 7.166667, 1e-6)
assert_near_equal(prob['y'], -7.833334, 1e-6)

In [ ]:
print(f"Number of errors: {len(comp.raised_eval_errors)}")
print(f"Iterations:{comp.raised_eval_errors}")

In [ ]:
assert(len(comp.raised_eval_errors) == 1)

In this case we can see that we raised a single AnalysisError.  We can again find evidence of SNOPT encountering this evaluation error in the `SNOPT_print.out` file, but still finding the solution. For SNOPT, we are looking for the `D` code at the end of an iteration. Here again we just show a relevant subsection of the file:

In [ ]:
with open("SNOPT_print.out", encoding="utf-8", errors='ignore') as f:
    SNOPT_history = f.read()
beg = SNOPT_history.find("   Itns Major Minor")
end = SNOPT_history.find("Problem name", beg)
print(SNOPT_history[beg:end])

In [ ]:
count = 0

for line in SNOPT_history.split('\n'):
    if line.endswith(' D'):
        print(line)
        count = count + 1

print("\nNumber of times SNOPT encountered an evaluation error:", count)


```{Note}
Not all optimizers will respond as nicely to an AnalysisError as the two demonstrated here (`IPOPT` and `SNOPT`).  Some optimizers may fail to navigate around the bad region and find a solution at all.  Other may find an incorrect solution.  It is important to understand the capabilities of your chosen optimizer when working with a model that may raise an AnlysisError.
```
